In [1]:
import pandas as pd
import os
import numpy as np
import swifter
import re
import string

In [2]:
df = pd.read_csv(r'/home/rahul/Downloads/Womens Clothing E-Commerce Reviews.csv')

In [3]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [4]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [5]:
df.drop(columns=['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Recommended IND','Positive Feedback Count',
                 'Division Name', 'Class Name','Department Name'], axis = 0, inplace = True)

In [6]:
df

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5
...,...,...
23481,I was very happy to snag this dress at such a ...,5
23482,"It reminds me of maternity clothes. soft, stre...",3
23483,"This fit well, but the top was very see throug...",3
23484,I bought this dress for a wedding i have this ...,3


## Change the columns name

In [7]:
df.rename(columns = {'Review Text':'description_of_a_clothing_item','Rating':'ranked_list'}, inplace = True)

In [8]:
df.ranked_list.value_counts()

5    13131
4     5077
3     2871
2     1565
1      842
Name: ranked_list, dtype: int64

In [9]:
# Checking Null Value
df.isnull().sum()/len(df)*100

description_of_a_clothing_item    3.597888
ranked_list                       0.000000
dtype: float64

In [10]:
df.dropna(inplace = True)

In [11]:
df.isnull().sum()/len(df)*100

description_of_a_clothing_item    0.0
ranked_list                       0.0
dtype: float64

In [12]:
df.shape

(22641, 2)

In [13]:
# Checking Duplicate Value
df[df.duplicated()]

,description_of_a_clothing_item,ranked_list
10067,Perfect fit and i've gotten so many compliment...,5
11074,"Lightweight, soft cotton top and shorts. i thi...",5
12526,I purchased this and another eva franco dress ...,2
13354,Perfect fit and i've gotten so many compliment...,5
14275,I bought this shirt at the store and after goi...,5
21470,The sweater and skirt are so pretty! they're r...,5
21888,"Love, love these jeans. being short they come ...",5


In [14]:
df.drop_duplicates(inplace = True)

In [15]:
df.shape

(22634, 2)

## Text Preprocessing

In [16]:
class text_preprocess:
    
    def __init__(self):
        pass

    def convert_to_lower(self, text):
        return text.lower()

    def remove_emojis(self, text):
        text = re.sub(r"(?:\@|https?\://)\S+", " ", text) #remove links and mentions
        text = re.sub(r"<.*?>"," ",text)

        wierd_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emotions
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u'\U00010000-\U0010ffff'
            u"\u200d"
            u"\u2640-\u2642"
            u"\u2600-\u2B55"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\u3030"
            u"\ufe0f"
            u"\u2069"
            u"\u2066"
            u"\u200c"
            u"\u2068"
            u"\u2067"
            "]+", flags=re.UNICODE)

        rm_emoji = wierd_pattern.sub(r'', text)
        return rm_emoji

    def remove_html(self, text):
        html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
        rm_html = re.sub(html, ' ', text)
        return rm_html

    def remove_URL(self,text):
        url = re.compile(r'https?://\S+|www\.\S+')
        URL = url.sub(r' ', text)
        return URL
    
    def remove_non_ascii(self, text):
        return re.sub(r'[^\x00-\x7f]',r' ', text) # or ''.join([x for x in text if x in string.printable]) 
    
    
    def remove_numbers(self, text):
        number_pattern = r'\d+'
        without_number = re.sub(pattern=number_pattern, repl=" ", string=text)
        return without_number


    def remove_punctuation(self,text):
        return text.translate(str.maketrans(' ', ' ', string.punctuation))


    def remove_extra_white_spaces(self, text):
        single_char_pattern = re.compile(r'\s+[a-zA-Z]\s+')
        without_sc = re.sub(single_char_pattern, r" ", text)
#         without_sc = text.replace(' ', '')
        return without_sc


    def preprocessText(self,text):            
        return self.remove_extra_white_spaces(self.remove_non_ascii(self.remove_URL(self.remove_html(self.remove_punctuation(self.remove_numbers(self.remove_emojis(self.convert_to_lower(text))))))))

     

In [17]:
if __name__ == "__main__":    
    text_prpocess_obj = text_preprocess()
    df.description_of_a_clothing_item = df.description_of_a_clothing_item.swifter.apply(lambda x: text_prpocess_obj.preprocessText(x))

Pandas Apply:   0%|          | 0/22634 [00:00<?, ?it/s]

In [18]:
df.description_of_a_clothing_item

0        absolutely wonderful  silky and sexy and comfo...
1        love this dress  its sooo pretty happened to f...
2        i had such high hopes for this dress and reall...
3        i love love love this jumpsuit its fun flirty ...
4        this shirt is very flattering to all due to th...
                               ...                        
23481    i was very happy to snag this dress at such gr...
23482    it reminds me of maternity clothes soft stretc...
23483    this fit well but the top was very see through...
23484    i bought this dress for wedding have this summ...
23485    this dress in lovely platinum is feminine and ...
Name: description_of_a_clothing_item, Length: 22634, dtype: object

In [19]:
df

,description_of_a_clothing_item,ranked_list
0,absolutely wonderful silky and sexy and comfo...,4
1,love this dress its sooo pretty happened to f...,5
2,i had such high hopes for this dress and reall...,3
3,i love love love this jumpsuit its fun flirty ...,5
4,this shirt is very flattering to all due to th...,5
...,...,...
23481,i was very happy to snag this dress at such gr...,5
23482,it reminds me of maternity clothes soft stretc...,3
23483,this fit well but the top was very see through...,3
23484,i bought this dress for wedding have this summ...,3


## Tokenization

In [20]:
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

2023-05-19 17:29:29.196210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 17:29:30.448440: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-19 17:29:30.448669: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-19 17:29:30.448687: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [21]:
# Tokenizing the description_of_a_clothing_item base texts.
def tokenize(text):
    my_doc = nlp(text)
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
    return token_list   

In [22]:

df.description_of_a_clothing_item = df.description_of_a_clothing_item.swifter.apply(lambda x: tokenize(x))

Pandas Apply:   0%|          | 0/22634 [00:00<?, ?it/s]

In [23]:
df.description_of_a_clothing_item

0        [absolutely, wonderful,  , silky, and, sexy, a...
1        [love, this, dress,  , its, sooo, pretty, happ...
2        [i, had, such, high, hopes, for, this, dress, ...
3        [i, love, love, love, this, jumpsuit, its, fun...
4        [this, shirt, is, very, flattering, to, all, d...
                               ...                        
23481    [i, was, very, happy, to, snag, this, dress, a...
23482    [it, reminds, me, of, maternity, clothes, soft...
23483    [this, fit, well, but, the, top, was, very, se...
23484    [i, bought, this, dress, for, wedding, have, t...
23485    [this, dress, in, lovely, platinum, is, femini...
Name: description_of_a_clothing_item, Length: 22634, dtype: object

## Remove stopwords

In [24]:
def remove_stopwords(text):    
    filtered_sentence =[] 
    for word in text:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    return " ".join(filtered_sentence)

In [25]:
df.description_of_a_clothing_item = df.description_of_a_clothing_item.swifter.apply(lambda x: remove_stopwords(x))

Pandas Apply:   0%|          | 0/22634 [00:00<?, ?it/s]

In [26]:
df.description_of_a_clothing_item

0            absolutely wonderful   silky sexy comfortable
1        love dress   sooo pretty happened find store m...
2        high hopes dress wanted work initially ordered...
3        love love love jumpsuit fun flirty fabulous ti...
4        shirt flattering adjustable tie perfect length...
                               ...                        
23481    happy snag dress great price easy slip flatter...
23482    reminds maternity clothes soft stretchy shiny ...
23483    fit worked m glad able try store nt order onli...
23484    bought dress wedding summer cute unfortunately...
23485    dress lovely platinum feminine fits perfectly ...
Name: description_of_a_clothing_item, Length: 22634, dtype: object

## Lemmatization

In [27]:
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')


w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = WordNetLemmatizer()

words = set(nltk.corpus.words.words())
# words = nltk.word_tokenize(corpus)

class lemmatization:
    
    def __init__(self):
        pass
    
    def lemmatizing_space(self, text):   
        return " ".join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

[nltk_data] Downloading package words to /home/rahul/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rahul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/rahul/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [28]:
if __name__ == "__main__":
  lemmatization_obj = lemmatization()
  df.description_of_a_clothing_item = df.description_of_a_clothing_item.swifter.apply(lambda x: lemmatization_obj.lemmatizing_space(x))

Pandas Apply:   0%|          | 0/22634 [00:00<?, ?it/s]

In [29]:
df.description_of_a_clothing_item

0              absolutely wonderful silky sexy comfortable
1        love dress sooo pretty happened find store m g...
2        high hope dress wanted work initially ordered ...
3        love love love jumpsuit fun flirty fabulous ti...
4        shirt flattering adjustable tie perfect length...
                               ...                        
23481    happy snag dress great price easy slip flatter...
23482    reminds maternity clothes soft stretchy shiny ...
23483    fit worked m glad able try store nt order onli...
23484    bought dress wedding summer cute unfortunately...
23485    dress lovely platinum feminine fit perfectly e...
Name: description_of_a_clothing_item, Length: 22634, dtype: object

## Fasttext
FastText is an open-source, free, lightweight library that allows users to learn text representations and text classifiers.

** FastText, by Facebook Research, is a library for efficient learning of word representations and text classification. FastText supports supervised (classifications) and unsupervised (embedding) representations of words and sentences.**

In [30]:
#data split for train and test

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.1, random_state = 2529, stratify = df.ranked_list)
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [31]:
df

,description_of_a_clothing_item,ranked_list
0,absolutely wonderful silky sexy comfortable,4
1,love dress sooo pretty happened find store m g...,5
2,high hope dress wanted work initially ordered ...,3
3,love love love jumpsuit fun flirty fabulous ti...,5
4,shirt flattering adjustable tie perfect length...,5
...,...,...
23481,happy snag dress great price easy slip flatter...,5
23482,reminds maternity clothes soft stretchy shiny ...,3
23483,fit worked m glad able try store nt order onli...,3
23484,bought dress wedding summer cute unfortunately...,3


In [32]:
#Data prepare for fasttext format for train

train["ranked_format"] = 0
    
for i in train.index:
    train.loc[i, 'ranked_format'] = "__ranked_list__" +str(train.ranked_list[i])+" "+str(train.description_of_a_clothing_item[i])

In [33]:
train

,description_of_a_clothing_item,ranked_list,ranked_format
0,great quality product unique design,5,__ranked_list__5 great quality product unique ...
1,absolutely love dress quality perfection desig...,5,__ranked_list__5 absolutely love dress quality...
2,love simple stylish shirt fabric soft fit true...,5,__ranked_list__5 love simple stylish shirt fab...
3,saw sweater big llama face knew ordered eagerl...,3,__ranked_list__3 saw sweater big llama face kn...
4,gorgeous print purchased recommend love larger...,5,__ranked_list__5 gorgeous print purchased reco...
...,...,...,...
20365,love sweater gotten compliment comfortable uni...,5,__ranked_list__5 love sweater gotten complimen...
20366,skirt ladylike light air cherry red color beau...,5,__ranked_list__5 skirt ladylike light air cher...
20367,thought getting nice light weight white sweate...,3,__ranked_list__3 thought getting nice light we...
20368,like dress color great style cute casual perfe...,4,__ranked_list__4 like dress color great style ...


In [34]:
#Data prepare for fasttext format for train

test["ranked_format"] = 0
    
for i in test.index:
    test.loc[i, 'ranked_format'] = "__ranked_list__" +str(test.ranked_list[i])+" "+str(test.description_of_a_clothing_item[i])

In [35]:
test

,description_of_a_clothing_item,ranked_list,ranked_format
0,ordered color usual size eptites size smaller ...,5,__ranked_list__5 ordered color usual size epti...
1,suit fit like glove ve kid currently nursing b...,5,__ranked_list__5 suit fit like glove ve kid cu...
2,liked purchased color green wine color exactly...,5,__ranked_list__5 liked purchased color green w...
3,material horrible heavy cheap skirt flow appea...,1,__ranked_list__1 material horrible heavy cheap...
4,awesome tshirt m glad finally went sale eying ...,5,__ranked_list__5 awesome tshirt m glad finally...
...,...,...,...
2259,got compliment dress time wore know work offic...,4,__ranked_list__4 got compliment dress time wor...
2260,ve wanted vest past year find perfect material...,5,__ranked_list__5 ve wanted vest past year find...
2261,concept sweater pretty usual size xl fit perfe...,4,__ranked_list__4 concept sweater pretty usual ...
2262,shirt adorable butit nt work m smaller gal lar...,2,__ranked_list__2 shirt adorable butit nt work ...


In [36]:
import fasttext

In [37]:
train.ranked_format.to_csv('fasttext_train.txt',index=None,header=None)

In [38]:
test.ranked_format.to_csv('fasttext_test.txt',index=None,header=None)

In [39]:
#train classifier model
model = fasttext.train_supervised('fasttext_train.txt', epoch= 250, lr=0.01, label_prefix= '__ranked_list__', dim=100)

Read 0M words
Number of words:  15697
Number of labels: 5
Progress: 100.0% words/sec/thread: 1672910 lr:  0.000000 avg.loss:  0.567465 ETA:   0h 0m 0s


In [40]:
#test fasttext model
model.test('fasttext_test.txt')

(2264, 0.5980565371024735, 0.5980565371024735)

In [41]:
result = model.test('fasttext_train.txt')
validation = model.test('fasttext_test.txt')

In [42]:
# DISPLAY ACCURACY OF TRAINED MODEL
text_line =  "accuracy:" + str(result[1])  +  ",validation:" + str(validation[1]) + '\n' 
print(text_line)

accuracy:0.8782523318605793,validation:0.5980565371024735



In [43]:
model.predict('fabric incredibly soft good weight pretty warm jacket warm sweatshirt fit close body typical bomber jacket fashion m lb went large fit find wishing bomber jacket tad longer exception nt fault jacket true style')

(('__ranked_list__5',), array([0.98276168]))

In [44]:
model.predict('loved way shirt looked forgiving flattering lightweight fabric nt withstand half day wear snagging time developing hole right center idea happened nt wearing near pulled shame staple wardrobe piece')

(('__ranked_list__2',), array([0.69800901]))

In [45]:
model.predict('poor quality material look thinner cheaper pictured')

(('__ranked_list__1',), array([0.75059617]))

In [46]:
model.predict('normally wear large medium review ordered med ve worn small love m glad ordered')

(('__ranked_list__5',), array([0.98638785]))

In [47]:
model.predict('shirt adorable butit nt work m smaller gal lar')

(('__ranked_list__2',), array([0.65594465]))